In [111]:
import time
import itertools
import string
import random
import Levenshtein
import networkx as nx
from scipy.sparse import coo_matrix
from sys import argv
from math import factorial
random.seed(1)
start_time = time.clock()

minimum_aa = 4
maximum_aa = 20
min_seqs = 0
max_seqs = 50000

In [109]:
print 'Number of iterations will be %d'%(factorial(max_seqs)/2/(factorial(max_seqs-2)))

Number of iterations will be 12497500


In [22]:
import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext(master='local[*]')

In [112]:
string_list = []

for i in range(min_seqs, max_seqs):
    random_integer = random.randrange(minimum_aa, maximum_aa)
    s = ''.join(random.choice('ACDEFGHIKLMNPQRSTVWY') for _ in range(random_integer))
    string_list.append(s)

d = {}

for i, s in enumerate(string_list):
    d[s] = i

In [113]:
strings_rdd = sc.parallelize(string_list)

In [114]:
strings_b = sc.broadcast(string_list)
d_b = sc.broadcast(d)

In [118]:
def generate_matrix_elements(iterator):
    strings = strings_b.value
    string_map = d_b.value
    
    for my_string in iterator: 
        for string in strings:
            ld = Levenshtein.distance(my_string, string)
            if ld < 2 and ld > 0: 
                row = string_map[my_string]
                col = string_map[string]
                yield (row, col, float(ld))

In [119]:
mat = strings_rdd.mapPartitions(generate_matrix_elements)

In [117]:
%time mat.count()

CPU times: user 15.1 ms, sys: 7.06 ms, total: 22.1 ms
Wall time: 5min 22s


5436

In [73]:
def get_matrix_elements(string_list):
    levenshtein_list = []
    rows = []
    cols = []

    for a, b in itertools.combinations(string_list, 2):
        levenshtein_distance = Levenshtein.distance(a, b)
        if levenshtein_distance < 2:
            levenshtein_list.append(1)
            rows.append(d[a])
            cols.append(d[b])
    return levenshtein_list, rows, cols

In [74]:
%lprun -f get_matrix_elements lev, rows, cols = get_matrix_elements(string_list)

In [92]:
%time lev, rows, cols = get_matrix_elements(string_list)

CPU times: user 5.99 s, sys: 244 ms, total: 6.24 s
Wall time: 6.05 s


In [ ]:
nx.write_gml(g, "g_500K.gml")


text_file = open("output_500K.txt", "w")
text_file.write(' '.join(('Number of generated sequences:', str(len(string_list)), 'sequences'))) 
text_file.write(' '.join(('\nTime calculating the Levenshtein distance:', str(dist_time), 'seconds')))
text_file.write(' '.join(('\nTime calculating the graph:', str(graph_time), 'seconds')))
text_file.write(' '.join(('\nTotal pipeline time:', str(total_time), 'seconds')))
text_file.write('\nSuccessful finish')
text_file.close()